In [1]:
import pandas as pd 
import numpy as np                     # For mathematical calculations 
import seaborn as sns                  # For data visualization 
import matplotlib.pyplot as plt  # For plotting graphs 
%matplotlib inline 
import warnings   # To ignore any warnings warnings.filterwarnings("ignore")
from sklearn.metrics import mean_absolute_error,mean_squared_error
import datetime as dt
from sklearn.model_selection import train_test_split as tts
from sklearn.linear_model import LinearRegression as LR1
from pandas.core.common import SettingWithCopyWarning

warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)

In [2]:
df=pd.read_csv("Titan.csv")
#print(df)
df1 = df.copy()
df1['conv_date'] = pd.to_datetime(df['Date'])
df1['conv_date'] = df1['conv_date'].map(dt.datetime.toordinal)
df1

,Date,Open,High,Low,Close,Volume,conv_date
0,2020-06-03,989.000000,1005.000000,979.700012,984.349976,4194237,737579
1,2020-06-04,970.099976,977.450012,942.349976,971.250000,5293647,737580
2,2020-06-05,977.000000,995.000000,967.000000,990.950012,3136137,737581
3,2020-06-08,1009.000000,1050.000000,1006.049988,1025.000000,7033675,737584
4,2020-06-09,1037.000000,1047.000000,989.250000,997.099976,17283990,737585
...,...,...,...,...,...,...,...
246,2021-05-28,1595.000000,1612.250000,1570.550049,1577.000000,1028675,737938
247,2021-05-31,1570.650024,1608.250000,1569.000000,1596.250000,1525715,737941
248,2021-06-01,1608.000000,1612.000000,1589.300049,1591.400024,1111933,737942
249,2021-06-02,1581.000000,1596.750000,1564.199951,1585.699951,1859258,737943


In [3]:
def p_d(predictdate,df1):
    b=len(df1)
    c1=df1.iloc[b-1]['conv_date']
    #print(c1)
    c0=df1.iloc[0]['conv_date']
    #print(c0)
    if predictdate>c1:
        predictdate1=c1
        i=len(df1)
    elif predictdate<=c0:
        predictdate1=c0
        i=1
    else:
        i=len(df1)-1
        while(i!=0):
            if predictdate<=df1['conv_date'][i]:
                i=i-1
                continue
            if i==0:
                i=1
                predictdate1=c0
            break
        predictdate1=df1['conv_date'][i]
        
    return(predictdate1,i)

In [4]:
p = dt.datetime(2021, 6, 4)
p = p.toordinal()
p_d(p,df1)

(737944, 251)

In [5]:
def openPrice(predictdate):
    data=df1
    data=data.reset_index()
    data=data.drop(columns=["index"])

    data["prev_open"]=data["Open"]
    for i in data.index:
        if i==0:
            continue
        data['prev_open'][i]=data['Open'][i-1]

    data["prev_close"]=data["Close"]
    for i in data.index:
        if i==0:
            continue
        data['prev_close'][i]=data['Close'][i-1]
    data["prev_high"]=data["High"]
    for i in data.index:
        if i==0:
            continue
        data['prev_high'][i]=data['High'][i-1]
    data["prev_low"]=data["Low"]
    for i in data.index:
        if i==0:
            continue
        data['prev_low'][i]=data['Low'][i-1]
    data["prev_volume"]=data["Volume"]
    for i in data.index:
        if i==0:
            continue
        data['prev_volume'][i]=data['Volume'][i-1]
    df2 = data.iloc[1:]
    predictdate1,i=p_d(predictdate,df2) 


    X=df2.drop(columns=['Date','Close','Low','High','Volume','Open'])
    Y=df2['Open']
    x_train,x_test,y_train,y_test=tts(X,Y,train_size=0.8)
    classifierO=LR1()
    classifierO.fit(x_train,y_train)
    y_pred=classifierO.predict(x_test)
  #  a=df2[df2["conv_date"]==predictdate]
    p_open=float(df2['Open'].where(df2['conv_date'] == predictdate1)[i])
    #print(p_open)
    p_close=float(df2['Close'].where(df2['conv_date'] == predictdate1)[i])
    p_high=float(df2['High'].where(df2['conv_date'] == predictdate1)[i])
    p_low=float(df2['Low'].where(df2['conv_date'] == predictdate1)[i])
    p_volume=float(df2['Volume'].where(df2['conv_date'] == predictdate1)[i])
    #print(p_open,p_close,p_high,p_low,p_volume)
    score=classifierO.score(x_test,y_test)
    print("Your Opening Model has a score of:",score)
    open_price=predictionOpen(predictdate,p_open,p_close,p_high,p_low,p_volume,X,Y,classifierO)
    return open_price

def predictionOpen(date,openp,close,high,low,vol,X,Y,classifierO):
    
    x = np.zeros(len(X.columns))
    #print(X.columns)
    
    
    x[1]=openp
    x[2]=close
    x[3]=high
    x[4]=low
    x[5]=vol
    x[0]=date
    #print(x)
    return classifierO.predict([x])[0]

In [6]:
def highPrice(predictdate,openp):
    data=df1
    data=data.reset_index()
    data=data.drop(columns=["index"])

    data["prev_open"]=data["Open"]
    for i in data.index:
        if i==0:
            continue
        data['prev_open'][i]=data['Open'][i-1]

    data["prev_close"]=data["Close"]
    for i in data.index:
        if i==0:
            continue
        data['prev_close'][i]=data['Close'][i-1]
    data["prev_high"]=data["High"]
    for i in data.index:
        if i==0:
            continue
        data['prev_high'][i]=data['High'][i-1]
    data["prev_low"]=data["Low"]
    for i in data.index:
        if i==0:
            continue
        data['prev_low'][i]=data['Low'][i-1]
    data["prev_volume"]=data["Volume"]
    for i in data.index:
        if i==0:
            continue
        data['prev_volume'][i]=data['Volume'][i-1]
    df2 = data.iloc[1:]
    predictdate1,i=p_d(predictdate,df2) 


    X=df2.drop(columns=['Date','Close','Low','High','Volume'])
    Y=df2['High']
    x_train,x_test,y_train,y_test=tts(X,Y,train_size=0.8)
    classifierH=LR1()
    classifierH.fit(x_train,y_train)
    y_pred=classifierH.predict(x_test)
  #  a=df2[df2["conv_date"]==predictdate]
    p_open=float(df2['Open'].where(df2['conv_date'] == predictdate1)[i])
    #print(p_open)
    p_close=float(df2['Close'].where(df2['conv_date'] == predictdate1)[i])
    p_high=float(df2['High'].where(df2['conv_date'] == predictdate1)[i])
    p_low=float(df2['Low'].where(df2['conv_date'] == predictdate1)[i])
    p_volume=float(df2['Volume'].where(df2['conv_date'] == predictdate1)[i])
    #print(p_open,p_close,p_high,p_low,p_volume)
    score=classifierH.score(x_test,y_test)
    print("Your Day High Model has a score of:",score)
    high_price=predictionHigh(predictdate,openp,p_open,p_close,p_high,p_low,p_volume,X,Y,classifierH)
    return high_price

def predictionHigh(date,open1,openp,close,high,low,vol,X,Y,classifierH):
    
    x = np.zeros(len(X.columns))
    #print(X.columns)
    
    x[0]=open1
    x[1]=date
    x[2]=openp
    x[3]=close
    x[4]=high
    x[5]=low
    x[6]=vol
    
    
    return classifierH.predict([x])[0]

In [7]:
def lowPrice(predictdate,openp):
    data=df1
    data=data.reset_index()
    data=data.drop(columns=["index"])

    data["prev_open"]=data["Open"]
    for i in data.index:
        if i==0:
            continue
        data['prev_open'][i]=data['Open'][i-1]

    data["prev_close"]=data["Close"]
    for i in data.index:
        if i==0:
            continue
        data['prev_close'][i]=data['Close'][i-1]
    data["prev_high"]=data["High"]
    for i in data.index:
        if i==0:
            continue
        data['prev_high'][i]=data['High'][i-1]
    data["prev_low"]=data["Low"]
    for i in data.index:
        if i==0:
            continue
        data['prev_low'][i]=data['Low'][i-1]
    data["prev_volume"]=data["Volume"]
    for i in data.index:
        if i==0:
            continue
        data['prev_volume'][i]=data['Volume'][i-1]
    df2 = data.iloc[1:]
    predictdate1,i=p_d(predictdate,df2) 


    X=df2.drop(columns=['Date','Close','Low','High','Volume'])
    Y=df2['Low']
    x_train,x_test,y_train,y_test=tts(X,Y,train_size=0.8)
    classifierH=LR1()
    classifierH.fit(x_train,y_train)
    y_pred=classifierH.predict(x_test)
  #  a=df2[df2["conv_date"]==predictdate]
    p_open=float(df2['Open'].where(df2['conv_date'] == predictdate1)[i])
    #print(p_open)
    p_close=float(df2['Close'].where(df2['conv_date'] == predictdate1)[i])
    p_high=float(df2['High'].where(df2['conv_date'] == predictdate1)[i])
    p_low=float(df2['Low'].where(df2['conv_date'] == predictdate1)[i])
    p_volume=float(df2['Volume'].where(df2['conv_date'] == predictdate1)[i])
    #print(p_open,p_close,p_high,p_low,p_volume)
    score=classifierH.score(x_test,y_test)
    print("Your Day Low Model has a score of:",score)
    low_price=predictionLow(predictdate,openp,p_open,p_close,p_high,p_low,p_volume,X,Y,classifierH)
    return low_price
def predictionLow(date,open1,openp,close,high,low,vol,X,Y,classifierL):
    
    x = np.zeros(len(X.columns))
   # print(X.columns)
    
    x[0]=open1
    x[1]=date
    x[2]=openp
    x[3]=close
    x[4]=high
    x[5]=low
    x[6]=vol
    
    
    return classifierL.predict([x])[0]

In [8]:
#company=input("Enter Symbol of company:")
day=int(input("Enter Day:"))
month=int(input("Enter Month:"))
year=int(input("Enter Year:"))
predictdate = dt.datetime(year, month, day)
predictdate = predictdate.toordinal()
openp=openPrice(predictdate)

high=highPrice(predictdate,openp)
low=lowPrice(predictdate,openp)
print("The predicted Open Price is:",openp)
print("The predicted high Price is:",high)
print("The predicted low Price is:",low)

Enter Day:14
Enter Month:6
Enter Year:2021
Your Opening Model has a score of: 0.9977157713497952
Your Day High Model has a score of: 0.9964716884417532
Your Day Low Model has a score of: 0.9954772765763304
The predicted Open Price is: 1674.7640219540954
The predicted high Price is: 1699.3870129613133
The predicted low Price is: 1649.6552446009337
